In [1]:
import pandas as pd
import numpy as np

## Выгрузить данные и вывести первые 10 строк

In [80]:
df = pd.read_csv('orders.csv')

In [4]:
df.head(10)

,date,vendor_id,chain_id,city_id,spec,successful_orders,fail_orders
0,2019-07-02,40065,7501,23,Суши,54.0,1.0
1,2019-08-27,48058,33226,23,Шаурма,0.0,0.0
2,2019-09-25,35430,26220,25,Пицца,2.0,0.0
3,2019-09-21,56553,38601,23,Шашлыки,1.0,0.0
4,2019-09-21,43919,30984,25,Суши,6.0,0.0
5,2019-06-13,53006,36214,26,Грузинская,2.0,0.0
6,2019-07-07,28982,20651,24,Суши,0.0,1.0
7,2019-08-01,45472,30409,24,Бургеры,37.0,4.0
8,2019-07-03,65358,43034,23,Русская,1.0,0.0
9,2019-09-19,60878,41231,23,Русская,4.0,0.0


## Сколько в датафрейсе колонок типа float64?

In [10]:
df.select_dtypes(include=['float64']).shape[1]

2

## Чему ровно произведение кол-ва наблюдений на кол-во колонок в датафрейме?

In [11]:
df.shape[0] * df.shape[1]

672826

## Сколько уникальных городов в датафрейме?

In [22]:
df.city_id.drop_duplicates().size

4

## Сколько успешных заказов в среднем получали рестораны в день?
Здесь я подрузамеваю, что нужно узнать среднее кол-во успешных заказов сделанных всеми ресторанами за день 

In [29]:
df.groupby(by='date').sum().successful_orders.mean()

7051.884297520661

## Кол-во ресторанов со специализацией "Рыба"? 

In [35]:
df[df.spec == 'Рыба'].vendor_id.drop_duplicates().size

18

## Сколько у ресторана 40065 было дней с кол-вом успешных заказов менее 20?

In [49]:
df_40065 = df[df.vendor_id == 40065]
(df_40065.groupby(by='date').successful_orders.sum() < 20).sum()

45

## Сколько заказов в среднем получили рестораны за рассматриваемый период времени?

In [52]:
df.groupby(by='vendor_id').successful_orders.sum().mean()

555.1581001951854

## Город с самым маленьким кол-вом успешных заказов за весь период?

In [148]:
suc_orders = df.groupby(by='city_id')[['successful_orders']].sum().reset_index()
suc_orders.iloc[suc_orders['successful_orders'].argmin()]

city_id                  24.0
successful_orders    114977.0
Name: 1, dtype: float64

## Какая специализация имеет самую высокую долю неуспешных заказов за весь период времени?

In [163]:
orders_info = df.groupby(by='spec').successful_orders.sum().reset_index()
orders_info = pd.merge(orders_info, df.groupby(by='spec').fail_orders.sum(), on='spec')
orders_info['fail_orders_ratio'] = orders_info.fail_orders / (orders_info.successful_orders + orders_info.fail_orders)
orders_info.iloc[orders_info.fail_orders_ratio.argmax()]

spec                 Вегетарианское меню
successful_orders                      5
fail_orders                           24
fail_orders_ratio               0.827586
Name: 1, dtype: object

## Какая из сетей (chain_id) показала максимальный абсолютный рост за август 2019 года по сравнению с июлем 2019 года? Учитывать только те сети, у которых заказы были в июне и в августе
К сожалению я ошибся в формате даты во время контрольной, написав вместо 2019-08-01 -> 2019-08-1 что естественно изменило мою выборку. Здесь привожу исправленную версию.

In [180]:
df_aug = df[(df['date'] < '2019-09-01') & (df['date'] >= '2019-08-01')]
df_june = df[(df['date'] < '2019-08-01') & (df['date'] >= '2019-07-01')]
df_aug_suc = df_aug.groupby(by='chain_id').successful_orders.sum().reset_index()
df_june_suc = df_june.groupby(by='chain_id').successful_orders.sum().reset_index()
df_month_statistics = pd.merge(df_june_suc, df_aug_suc, on='chain_id', how='inner', suffixes=('_june', '_august'))
df_month_statistics['absolute_growth'] = df_month_statistics.successful_orders_august - \
                                                df_month_statistics.successful_orders_june
df_month_statistics.iloc[df_month_statistics.absolute_growth.argmax()]

chain_id                    29925.0
successful_orders_june      11167.0
successful_orders_august    12561.0
absolute_growth              1394.0
Name: 201, dtype: float64